<a href="https://colab.research.google.com/github/epare2496-commits/labo2025v/blob/main/src/rpart/EP%20299_TareaHogar_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea para el Hogar 02

Esta Tarea para el Hogar 02 se entrega el final de la segunda clase
<br> se espera de usted que intente avanzar con los desafios propuestos y que los traiga terminados para la Clase 03, ya que se analizarán los resultados

##  1. Ensembles de Modelos

Vea el siguiente video [BBC - The Code - The Wisdom of the Crowd](https://www.youtube.com/watch?v=iOucwX7Z1HU)    ( 5 min)


Lea los siguientes artículos


*   [The Wisdom of Crowds (Vox Populi) by Francis Galton](https://www.all-about-psychology.com/the-wisdom-of-crowds.html)  (10 min)
*   [A Gentle Introduction to Ensemble Learning](https://machinelearningmastery.com/what-is-ensemble-learning/)  (10 min)





---



##  2.  Zero2Hero   primera parte
Se han lanzado los primeros fascículos coleccionables llamados "from Zero to Hero" que muy detalladamente, paso a paso enseñan todo lo necesario de R para entender los scripts oficiales de la asignatura.
Están en el repositorio oficial de la asignatura, carpeta  **src/zero2hero**



---



## 3.  Grid Search

Busque en internet el precido significado de los hiperparámetros de la librería **rpart**  que está implementando el algoritmo **CART**  Classification and Regression Trees  propuesto en el año 1984 por Leo Brieman:

*   cp
*   maxdepth
*   minsplit
*   minbucket

Entienda que valores es razonable tome cada hiperparámetro,  en particular profundice en el hiperparámetro  **cp**  y la posibilidad que tome valores negativos.  Es válido consultar a su amigo de *capacidades especiales*  ChatGPT


En las siguientes celdas a un notebook incompleto, un esqueleto de codigo brindado a modo de facilitarle la tarea de codeo y permitir que su valiosa cognición se concentre temas conceptuales de Ciencia de Datos

Modifiquelo agregando loops para que recorra TODOS los hiperparámetros de rpart  < cp, maxdepth, minsplit, minbucket >, y luego póngalo a correr. Recuerde cambiar por SU semilla
Tenga muy presente la granularidad que eligirá para cada hiperparámetro.

### Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



limpio el ambiente de R

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657245,35.2,1454460,77.7,1326122,70.9
Vcells,1220272,9.4,8388608,64.0,1975128,15.1


In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")
if (!require("primes")) install.packages("primes")
require("primes")

Loading required package: data.table

Loading required package: rpart

Loading required package: parallel

Loading required package: primes

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘primes’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: primes



Aqui debe poner SU semiila primigenia

In [3]:
PARAM <- list()
# reemplazar por su primer semilla
PARAM$semilla_primigenia <- 100009
PARAM$qsemillas <- 1

PARAM$training_pct <- 70L  # entre  1L y 99L

# elegir SU dataset comentando/ descomentando
PARAM$dataset_nom <- "~/datasets/dataset_pequeno.csv"

In [4]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)
#   crea una particion 70, 30

particionar <- function(data, division, agrupa = "", campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}


In [5]:
ArbolEstimarGanancia <- function(semilla, training_pct, param_basicos, PARAM) {
  # (Se asume que las funciones 'particionar' y 'dataset' están disponibles)

  # 1. Particionar el dataset
  particionar(dataset, division = c(training_pct, 100L - training_pct), agrupa = "clase_ternaria", seed = semilla)

  # 2. Generar el modelo (rpart)
  modelo <- rpart("clase_ternaria ~ .", data = dataset[fold == 1], xval = 0, control = param_basicos)

  # 3. Aplicar el modelo a testing
  df_testing <- dataset[fold == 2]
  prediccion <- predict(modelo, df_testing, type = "prob")

  # --- Cálculo de la Ganancia y Métricas (Basado en Umbral 0.025) ---

  # Clientes que el modelo DECIDIÓ CONTACTAR (Prob > 0.025)
  decision_contacto <- prediccion[, "BAJA+2"] > 0.025

  # Realidad de los clientes en testing
  realidad_baja2 <- df_testing$clase_ternaria == "BAJA+2"

  # Componentes de la Matriz de Confusión:
  VP <- sum(decision_contacto & realidad_baja2)       # True Positives (Ganancia 117000)
  FP <- sum(decision_contacto & !realidad_baja2)      # False Positives (Pérdida -3000)
  FN <- sum(!decision_contacto & realidad_baja2)      # False Negatives (Oportunidad Perdida)

  # Cálculo de Métricas
  Precision <- ifelse((VP + FP) > 0, VP / (VP + FP), 0) # Si se contacta a alguien (VP + FP) > 0
  Recall <- ifelse((VP + FN) > 0, VP / (VP + FN), 0)     # Si hay casos BAJA+2 reales (VP + FN) > 0

  # 4. Cálculo de la ganancia en testing (Se mantiene el código original)
  ganancia_test <- VP * 117000 + FP * (-3000)

  # 5. Normalizar la ganancia
  prop_testing <- (100 - training_pct) / 100
  ganancia_test_normalizada <- ganancia_test / prop_testing

  # --- Métricas de Diagnóstico ADICIONALES (Reutilizadas) ---
  nodos_terminales <- sum(modelo$frame$var == "<leaf>")
  profundidad_maxima <- max(floor(log2(as.numeric(row.names(modelo$frame[modelo$frame$var == "<leaf>",])))))
  error_relativo_min <- min(modelo$cptable[, "rel error"])
  imp_vars <- modelo$variable.importance
  variable_mas_imp <- if (length(imp_vars) > 0) names(imp_vars)[1] else "N/A"

  # 6. Devolver el resultado
  return(
    c(list(
      "semilla" = semilla
    ),
      param_basicos,
      list(
        "ganancia_test_normalizada" = ganancia_test_normalizada,
        "nodos_terminales" = nodos_terminales,
        "profundidad_maxima" = profundidad_maxima,
        "precision" = Precision,
        "recall" = Recall,
        "error_relativo_min" = error_relativo_min,
        "variable_mas_imp" = variable_mas_imp
      )
    )
  )
}

In [6]:
ArbolesMontecarlo <- function(semillas, param_basicos) {

  # la funcion mcmapply  llama a la funcion ArbolEstimarGanancia
  #  tantas veces como valores tenga el vector  PARAM$semillas
  salida <- mcmapply(ArbolEstimarGanancia,
    semillas, # paso el vector de semillas
    MoreArgs = list(PARAM$training_pct, param_basicos), # aqui paso el segundo parametro
    SIMPLIFY = FALSE,
    mc.cores = detectCores()
  )

  return(salida)
}


In [7]:
# carpeta de trabajo
# por favor cambiar numero de experimento si se cambia el loop principal
setwd("/content/buckets/b1/exp")
experimento <- "HT2909"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [8]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [9]:

# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, PARAM$qsemillas )


In [10]:
# genero la data.table donde van los resultados detallados del Grid Search
# un registro para cada combinacion de < semilla, parametros >

if(file.exists("gridsearch_detalle.txt")){
  tb_grid_search_detalle <- fread("gridsearch_detalle.txt")
}else{
  tb_grid_search_detalle <- data.table(
    semilla = integer(),
    cp = numeric(),
    maxdepth = integer(),
   minsplit = integer(),
    minbucket = integer(),
    ganancia_test = numeric()
  )
}

nrow( tb_grid_search_detalle )

[1] 0

Esta es la parte del código que usted debe expandir a TODOS los hiperparámetros de rpart,
<br>ya que actualmente apenas recorre  maxdepth y  minsplit  dejando fijos  cp=-0.5  y minbucket=5

In [12]:
# @title

# itero por los loops anidados para cada hiperparametro

iter <- 0

# LOOP 1: Complexity Parameter (cp)
for (vcp in c(-0.01, 0.01, 0.05)) {

  # LOOP 2: Minimum Bucket Size (minbucket)
  for (vmin_bucket in c(65, 75, 90)) {

    # LOOP 3: Maximum Depth (maxdepth)
    for (vmax_depth in c( 8, 10, 15, 30)) {

      # LOOP 4: Minimum Split Size (minsplit)
      for (vmin_split in c(500,800, 1200, 2000, 1400)) {

        # Notar que la iteración total es ahora 3 * 3 * 6 * 5 = 270
        iter <- iter + 1
        cat( iter, " " )
        flush.console()
        if( iter*PARAM$qsemillas < nrow(tb_grid_search_detalle)+1 ) next

        # Definición de Hiperparámetros
        param_basicos <- list(
          "cp" = vcp,             # complejidad
          "maxdepth" = vmax_depth, # profundidad maxima del arbol
          "minsplit" = vmin_split, # tamaño minimo de nodo para split
          "minbucket" = vmin_bucket # minima cantidad de registros en una hoja
        )

        # Un solo llamado, con la semilla 17
        ganancias <- ArbolesMontecarlo(PARAM$semillas, param_basicos)

        # agrego a la tabla
        tb_grid_search_detalle <- rbindlist(
          list( tb_grid_search_detalle,
                rbindlist(ganancias) ), fill = TRUE
        )

      } # Fin Loop minsplit

    } # Fin Loop maxdepth

    # IMPORTANTE: El guardado ocurre aquí, después de completar todas las combinaciones de minsplit y maxdepth
    # para un valor específico de CP y MINBUCKET.

    # grabo cada vez TODA la tabla en el loop mas externo
    fwrite( tb_grid_search_detalle,
            file = "gridsearch_detalle.txt",
            sep = "\t" )

  } # Fin Loop minbucket

} # Fin Loop cp

1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180  

In [13]:
fwrite( tb_grid_search_detalle,
   file = "gridsearch_detalle.txt",
   sep = "\t"
)

In [14]:
# cantidad de registros de la tabla
nrow(tb_grid_search_detalle)

[1] 180

In [15]:
# muestro la tabla
tb_grid_search_detalle

semilla,cp,maxdepth,minsplit,minbucket,ganancia_test,ganancia_test_normalizada,nodos_terminales,profundidad_maxima,precision,recall,error_relativo_min,variable_mas_imp
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
842473,-0.01,8,500,65,NA,52070000,64,8,0.06576887,0.5614973,1,ctrx_quarter
842473,-0.01,8,800,65,NA,51480000,59,8,0.06609195,0.5534759,1,ctrx_quarter
842473,-0.01,8,1200,65,NA,52730000,53,8,0.06037027,0.6016043,1,ctrx_quarter
842473,-0.01,8,2000,65,NA,51360000,41,8,0.05788934,0.6042781,1,ctrx_quarter
842473,-0.01,8,1400,65,NA,51870000,49,8,0.05936920,0.5989305,1,ctrx_quarter
842473,-0.01,10,500,65,NA,50350000,102,10,0.05611867,0.6069519,1,ctrx_quarter
842473,-0.01,10,800,65,NA,51660000,91,10,0.05670103,0.6176471,1,ctrx_quarter
842473,-0.01,10,1200,65,NA,52990000,77,10,0.05417309,0.6577540,1,ctrx_quarter
842473,-0.01,10,2000,65,NA,50980000,56,10,0.05733130,0.6042781,1,ctrx_quarter


In [16]:
# genero y grabo el resumen
tb_grid_search <- tb_grid_search_detalle[,
  list( "ganancia_mean" = mean(ganancia_test),
    "qty" = .N ),
  list( cp, maxdepth, minsplit, minbucket )
]


In [17]:
# ordeno descendente por ganancia
setorder( tb_grid_search, -ganancia_mean )


In [18]:
# veo los 10 mejores hiperparámetros
tb_grid_search[1:10]

cp,maxdepth,minsplit,minbucket,ganancia_mean,qty
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
-0.01,8,500,65,NA,1
-0.01,8,800,65,NA,1
-0.01,8,1200,65,NA,1
-0.01,8,2000,65,NA,1
-0.01,8,1400,65,NA,1
-0.01,10,500,65,NA,1
-0.01,10,800,65,NA,1
-0.01,10,1200,65,NA,1
-0.01,10,2000,65,NA,1


In [19]:
# genero un id a la tabla
tb_grid_search[, id := .I ]

fwrite( tb_grid_search,
  file = "gridsearch.txt",
  sep = "\t"
)


# 4.  Análisis de resultados de Grid Search

La salida de la corrida anterior queda en ~/buckets/b1/exp/HT2900  que corresponde a su Google Drive
<br>HT significa Hyperparameter Tuning
<br>El Grid Search es un método de fuerza bruta de un altísimo costo computacional.
<br>Queremos ver si es posible crear un algoritmo de optimización de hiperparámetros que se ahorre recorrer ciertas porciones muy malas del espacio de búsqueda. Algo del estilo “cada vez que pruebo una combinación de hiperparámetros donde  cp > 1 , la ganancia es muy mala, con lo cual ni vale la pena perder el tiempo explorando en esa region”


<br>Levante el archivo de salida gridsearch.txt  a una planilla tipo Excel y analícelo detenidamente
<br>Ordene por ganancia_mean descendente
<br>
<br>El de mayor ganancia_mean  decimos que es el primero del ranking
En Zulip, correspondiente channel  #Tarea Hogar 02 , topic Analisis Grid Search   intente contestar estas preguntas:

* ¿Qué combinaciones de hiperparámetros poseen una ganancia muy buena?
* ¿Hay algun hiperparámetro que para cierto valor siempre genera una ganancia muy mala, a independientemente de lo que valgan los otros hiperparámetros ?
* ¿Que combinaciones de hiperparámetros es pésima y hubiera sido bueno ahorrarse esas corridas ?

( tiempo estimado 30 minutos, dificultad media )